In [1]:
import pandas as pd

#Daten einlesen
data_orig = pd.read_parquet('../src/data/interim/data_numeric.parquet')

In [2]:
#Zeilen mit leeren Werten löschen
#data_orig = data_orig.dropna()
#Duplikate entfernen
print('zeilen vor Duplikatentfernung: ', len(data_orig))
data_orig = data_orig[data_orig.duplicated()==False]
print('zeilen nach Duplikatentfernung: ', len(data_orig))

zeilen vor Duplikatentfernung:  26601
zeilen nach Duplikatentfernung:  26561


In [17]:
data_orig.columns

Index(['look_at_dates', 'age', 'age_stop_edu', 'household_size', 'cntrylat',
       'cntrylon', 'best_before_meaning_map', 'validity_meaning_map',
       'work_scale', 'population_density', 'salary',
       'no_date_spaghetti_throw_away', 'gender_Female'],
      dtype='object')

In [18]:
#2. modell erstellen
#Pipelines ZielSpalte nicht Transformer einegebn
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing  import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

pipeline = make_pipeline(
    ColumnTransformer([
        ('select_columns','passthrough',
         ['look_at_dates', 'age', 'age_stop_edu', 'household_size', 'cntrylat', 'cntrylon', 'best_before_meaning_map', 'validity_meaning_map',
          'work_scale', 'population_density', 'salary', 'gender_Female'])
    ]),
    SimpleImputer(),
    StandardScaler(),
    RandomForestClassifier(

        random_state=42
    )
)

In [19]:
#3Auswerten

from sklearn.model_selection import StratifiedKFold,cross_val_score
cv_splitter = StratifiedKFold(shuffle=True)
cross_val_score(pipeline,data_orig,data_orig['no_date_spaghetti_throw_away'], cv=cv_splitter,scoring='f1').mean()

0.32275453125022796

In [7]:
#4 Modell mit alen Daten trainieren
pipeline.fit(data_orig,data_orig['no_date_spaghetti_throw_away'])

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('select_columns',
                                                  'passthrough',
                                                  ['look_at_dates', 'age',
                                                   'age_stop_edu',
                                                   'household_size',
                                                   'best_before_meaning_map',
                                                   'validity_meaning_map'])])),
                ('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=7, n_estimators=2,
                                        random_state=42))])

In [8]:
#5 Modell speichern
import pickle
with open('pipeline_spaghetti.pkl','wb') as file:
    pickle.dump(pipeline,file)

In [9]:
with open('pipeline_spaghetti.pkl','rb') as file:
    pipeline = pickle.load(file)

In [10]:
#7: Daten zur Vorhersgae bestimmen
prediction_input = data_orig.sample(10)
prediction_input

,look_at_dates,age,age_stop_edu,household_size,cntrylat,cntrylon,best_before_meaning_map,validity_meaning_map,work_scale,population_density,salary,no_date_spaghetti_throw_away,gender_Female
15976,4.0,65,18.0,2.0,35.0000,33.00,1.0,1.0,0.0,2.0,0.1139,0,0
17561,4.0,82,18.0,2.0,59.0000,26.00,1.0,2.0,0.0,0.0,0.1139,0,0
12285,4.0,57,23.0,2.0,62.0000,15.00,2.0,0.0,2.0,2.0,0.5800,0,0
10878,4.0,53,23.0,3.0,39.5000,-8.00,2.0,2.0,3.0,2.0,0.8300,0,0
26024,3.0,63,18.0,3.0,45.1667,15.50,1.0,2.0,0.0,1.0,0.1139,0,1
9406,2.0,72,21.0,2.0,52.5000,5.75,2.0,1.0,0.0,0.0,0.1139,0,0
14159,4.0,18,NaN,2.0,54.0000,-2.00,0.0,0.0,0.0,0.0,0.0200,1,1
23970,4.0,29,27.0,2.0,48.6667,19.50,1.0,0.0,2.0,0.0,0.5000,0,1
25469,1.0,41,19.0,4.0,46.0000,15.00,1.0,1.0,2.0,1.0,0.2270,0,1
189,3.0,49,20.0,2.0,50.8333,4.00,2.0,2.0,2.0,0.0,0.2270,0,1


In [11]:
#8: Vorhersage machen
predictions = pipeline.predict(prediction_input)
predictions

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=uint8)

In [12]:
#9 Vorhersage und Input-Daten in Excel-Datei speichern
with pd.ExcelWriter('prediction_spa.xlsx', engine='openpyxl', mode='wa') as writer:
    prediction_input.to_excel(writer, sheet_name='Prediction Input')
    pd.Series(predictions).to_excel(writer, sheet_name='Predictions')